In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('C:/Users/AKHIL/Desktop/scrapper/diseases.db')
c = conn.cursor()
df = pd.read_sql_query('SELECT * FROM diseases',conn);

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [3]:
stop_words = stopwords.words('english')
stop_words = set(stop_words)
stop_words.add('(')
stop_words.add(')')
stop_words.add(':')
stop_words.add(',')
stop_words.add('.')
stop_words.add('a')
stop_words.add('the')
stop_words.add('you')
stop_words.add('may')
stop_words.add('symptoms')
stop_words.add('an')
stop_words.add('so')
stop_words.add('include')
stop_words = list(stop_words)

In [4]:
st_lis = []
all_words = []
for row in df["Symptoms"]:
    words = word_tokenize(row)
    words = set(words)
    words = [word.lower() for word in words]
    words = [word for word in words if not word in stop_words]
    all_words = all_words + words
    s = ''
    for word in words:
        s = s + word + ' '
    st_lis.append(s)
    

In [5]:
weights = nltk.FreqDist(all_words)

In [6]:
symptoms = pd.Series(st_lis)

In [7]:
df = df.drop(["Symptoms"],axis=1)

In [8]:
df["Symptoms"] = symptoms

In [9]:
df = pd.concat([df,df['Symptoms'].str.get_dummies(sep=' ')],axis=1)

In [10]:
u = list(df.columns)

In [11]:
x = u[0:33]

In [12]:
for r in x:
    u.remove(r)

In [13]:
X = pd.DataFrame(df,columns = u)
for f in u:
    X[f] = X[f]*(1/weights[f])

In [14]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean').fit(X)

In [23]:
xtest = X.iloc[26]
xtest = xtest.values.reshape(1, -1)
distances, indices = nbrs.kneighbors(xtest)
print(distances)
print(indices[0][:])

[[ 0.          1.2647138   1.34504627  1.41990437  1.71835317  1.71936369
   1.73578201  1.78201267  1.82725114  1.8559305 ]]
[ 26 102  69 110  84  35   7  78 118  55]


In [24]:
top_similar_diseases = [df.iloc[indice]["Name"] for indice in indices[0][:]]

In [25]:
top_similar_diseases

['Bone cancer',
 'Broken heart syndrome',
 'Heartburn',
 'Stomach polyps',
 'Dislocated elbow',
 'Microscopic colitis',
 'Astigmatism',
 'Ovarian cancer',
 'Severe acute respiratory syndrome (SARS)',
 'Sudden cardiac arrest']

In [18]:
df.iloc[26]["Symptoms"]

'swelling broken fatigue loss unintended bone near weight signs area pain tenderness cancer affected bone '

In [22]:
df.iloc[20]["Symptoms"]

'swollen often 39 skin elsewhere disease trunk lips swollen perhaps higher eyes conjunctivitis nodes 102.2 usually c without genital phases lasts extremely days extremely kawasaki part rash strawberry cracked dry neck tongue area f palms fever body discharge red three hands feet red lymph appear thick main soles irritability '